In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(
    'C:/Users/Diego/Desktop/UDEMY/metabase_for_udemy/data/sales_detail.csv', 
    encoding='latin1', 
    index_col=False
)

In [3]:
df = df.drop(columns=['status'])

In [4]:
purchase_states = {
    1: {"Pending"},    # Pendiente
    2: {"Confirmed"},  # Confirmado
    3: {"Shipped"},    # Enviado
    4: {"Delivered"},  # Entregado
    5: {"Returned"}    # Devuelto
}

allowed_states = ["Pending","Confirmed","Shipped","Delivered","Returned"]


In [5]:
# Extraer solo los valores (estados) del diccionario
states = list(purchase_states.values())

# Agregar la columna 'status' con un estado aleatorio para cada fila
df['status'] = np.random.choice(allowed_states, size=len(df))


In [6]:
# Crear un diccionario que mapea 'codcli' a 'name_complete'
name_dict = {
    1: 'Juan Carlos González',
    2: 'María Fernanda López',
    3: 'José Manuel Rodríguez',
    4: 'Ana Patricia Martínez',
    5: 'Carlos Eduardo Pérez',
    6: 'Luisa Margarita Ramírez',
    7: 'Pedro Antonio Gutiérrez',
    8: 'Rosa Elena Fernández',
    9: 'Jorge Alberto Vargas',
    10: 'Gabriela Mercedes Pineda',
    11: 'Daniel Esteban Morales',
    12: 'Camila Sofía Castillo',
    13: 'Ricardo Emilio Sánchez',
    14: 'Adriana Isabel Jiménez',
    15: 'Fernando Andrés Ortega',
    16: 'Valeria Beatriz Herrera',
    17: 'Oscar David Salinas',
    18: 'Margarita Susana Rivas',
    19: 'Diego Alejandro Núñez',
    20: 'Claudia Josefina Torres',
    21: 'Rafael Enrique Ruiz',
    22: 'Silvia Alejandra Flores',
    23: 'Andrés Javier Mendoza',
    24: 'Laura Beatriz Mejía'
}

In [7]:
# Asignar la columna 'name_complete' usando el mapeo del diccionario
df['name_complete'] = df['CODCLI'].map(name_dict)

In [8]:
df.head(2)

,CODCLI,CODART,Cliente,Pais,Comunidad autÃ³noma,Provincia,Zona,ArtÃ­culo,Serie,Nfactura,Fecha,Unidades,Precio,Costo,status,name_complete
0,3,1,CARLOS HUMBERTO GARCIA MERCADO,NICARAGUA,CARAZO,CARAZO,Centro,Bicicleta carrera,S2,1,41279,8,"551,02","418,78",Pending,José Manuel Rodríguez
1,3,2,CARLOS HUMBERTO GARCIA MERCADO,NICARAGUA,CARAZO,CARAZO,Centro,Bicicleta mountain infantil,S2,2,41280,22,"116,99","98,27",Shipped,José Manuel Rodríguez


In [9]:
# Sobrescribir el archivo CSV con el DataFrame actualizado
df.to_csv('./data/sales_detail.csv', index=False)